In [1]:
%load_ext autoreload
%autoreload

In [2]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import logging
logging.getLogger().setLevel(logging.INFO)

import copy
import multiprocessing as mp

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash

from config import config
from pof.loader.asset_model_loader import AssetModelLoader
from pof import Component, FailureMode, Task
from pof.interface.dashlogger import DashLogger
from pof.interface.layouts import *
from pof.interface.figures import update_condition_fig, update_cost_fig, update_pof_fig, make_inspection_interval_fig

In [3]:
comp = Component.demo()
fm = FailureMode.demo()

In [4]:
comp.mc_timeline(1000)
update_condition_fig(comp)

100%|██████████| 100/100 [00:04<00:00, 23.39it/s]


In [4]:
comp.mc_timeline(10)
make_inspection_interval_fig_drop(comp)

100%|██████████| 10/10 [00:00<00:00, 76.91it/s]


In [4]:
comp.mc_timeline(1000)
update_cost_fig(comp)

100%|██████████| 100/100 [00:04<00:00, 24.54it/s]


In [5]:
comp.mc_timeline(1000)
update_pof_fig(comp)

100%|██████████| 100/100 [00:05<00:00, 19.99it/s]


In [ ]:

def make_inspection_interval_fig_drop(
    local, t_min=0, t_max=10, step=1, n_iterations=10
):

    try:
        df = local.expected_inspection_interval(
            t_min=t_min, t_max=t_max, step=step, n_iterations=n_iterations
        )
        df = df.set_index("inspection_interval", drop=True)

        fig = go.Figure()

        for column in df.columns.to_list():
            fig.add_trace(
                go.Scatter(
                    x=df.index,
                    y=df[column],
                    name=column,
                    showlegend=False,
                )
            )

        fig.update_yaxes(automargin=True)
        fig.update_xaxes(automargin=True)

        # Add dropdown
        fig.update_layout(
            title="Risk v Cost at different Inspection Intervals",
            updatemenus=[
                go.layout.Updatemenu(
                    active=0,
                    buttons=list(
                        [
                            dict(
                                label="All",
                                method="update",
                                args=[
                                    {
                                        "visible": [
                                            True,
                                            True,
                                            True,
                                            True,
                                            True,
                                            True,
                                            True,
                                        ]
                                    },
                                    {
                                        "title": "Risk v Cost at different Inspection Intervals",
                                        "showlegend": False,
                                    },
                                ],
                            ),
                            dict(
                                label="Inspection",
                                method="update",
                                args=[
                                    {
                                        "visible": [
                                            True,
                                            False,
                                            False,
                                            False,
                                            False,
                                            False,
                                            False,
                                        ]
                                    },  # the index of True aligns with the indices of plot traces
                                    {
                                        "title": "Risk v Cost at different Inspection Intervals",
                                        "showlegend": False,
                                    },
                                ],
                            ),
                            dict(
                                label="On Condition Repair",
                                method="update",
                                args=[
                                    {
                                        "visible": [
                                            False,
                                            True,
                                            False,
                                            False,
                                            False,
                                            False,
                                            False,
                                        ]
                                    },
                                    {
                                        "title": "Risk v Cost at different Inspection Intervals",
                                        "showlegend": False,
                                    },
                                ],
                            ),
                            dict(
                                label="On Condition Replacement",
                                method="update",
                                args=[
                                    {
                                        "visible": [
                                            False,
                                            False,
                                            True,
                                            False,
                                            False,
                                            False,
                                            False,
                                        ]
                                    },
                                    {
                                        "title": "Risk v Cost at different Inspection Intervals",
                                        "showlegend": False,
                                    },
                                ],
                            ),
                            dict(
                                label="On Failure Replacement",
                                method="update",
                                args=[
                                    {
                                        "visible": [
                                            False,
                                            False,
                                            False,
                                            True,
                                            False,
                                            False,
                                            False,
                                        ]
                                    },
                                    {
                                        "title": "Risk v Cost at different Inspection Intervals",
                                        "showlegend": False,
                                    },
                                ],
                            ),
                            dict(
                                label="Risk Cost",
                                method="update",
                                args=[
                                    {
                                        "visible": [
                                            False,
                                            False,
                                            False,
                                            False,
                                            True,
                                            False,
                                            False,
                                        ]
                                    },
                                    {
                                        "title": "Risk v Cost at different Inspection Intervals",
                                        "showlegend": False,
                                    },
                                ],
                            ),
                            dict(
                                label="Direct Cost",
                                method="update",
                                args=[
                                    {
                                        "visible": [
                                            False,
                                            False,
                                            False,
                                            False,
                                            False,
                                            True,
                                            False,
                                        ]
                                    },
                                    {
                                        "title": "Risk v Cost at different Inspection Intervals",
                                        "showlegend": False,
                                    },
                                ],
                            ),
                            dict(
                                label="Total",
                                method="update",
                                args=[
                                    {
                                        "visible": [
                                            False,
                                            False,
                                            False,
                                            False,
                                            False,
                                            False,
                                            True,
                                        ]
                                    },
                                    {
                                        "title": "Risk v Cost at different Inspection Intervals",
                                        "showlegend": False,
                                    },
                                ],
                            ),
                        ]
                    ),
                    direction="down",
                    pad={"r": 10, "t": 10},
                    showactive=True,
                    x=0.5,
                    xanchor="right",
                    y=1.2,
                    yanchor="top",
                ),
            ],
        )

    except:
        fig = go.Figure(
            layout=go.Layout(
                title=go.layout.Title(text="Error Producing Inspection Interval")
            )
        )

    return fig

In [12]:
cond_name = "cond name"
"".join([x[0] for x in cond_name.split(" ")])

'cn'